# 7.2 数据转换

本章到目前为止介绍的都是数据的重排。另一类重要操作则是过滤、清理以及其他的转换工作。

## 移除重复数据

DataFrame中出现重复行有多种原因。下面就是一个例子：

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]}
                   )

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame的duplicated方法返回一个布尔型Series，表示各行是否是重复行（前面出现过的行）：

In [3]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

还有一个与此相关的drop_duplicates方法，它会返回一个DataFrame，重复的数组会标为False：

In [4]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。假设我们还有一列值，且只希望根据k1列过滤重复项：

In [5]:
data['v1'] = range(7)

data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [6]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated和drop_duplicates默认保留的是第一个出现的值组合。传入keep='last'则保留最后一个：

In [7]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [8]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## 利用函数或映射进行数据转换

对于许多数据集，你可能希望根据数组、Series或DataFrame列中的值来实现转换工作。我们来看看下面这组有关肉类的数据：

In [9]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]}
                   )

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


假设你想要添加一列表示该肉类食物来源的动物类型。我们先编写一个不同肉类到动物的映射：

In [10]:
meat_to_animal = {
  'bacon':       'pig',
  'pulled pork': 'pig',
  'pastrami':    'cow',
  'corned beef': 'cow',
  'honey ham':   'pig',
  'nova lox':    'salmon'
}

Series的map方法可以接受一个函数或含有映射关系的字典型对象，但是这里有一个小问题，即有些肉类的首字母大写了，而另一些则没有。因此，我们还需要使用Series的str.lower方法，将各个值转换为小写：

In [11]:
lowercased = data['food'].str.lower()

lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [12]:
data['animal'] = lowercased.map(meat_to_animal)

data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


我们也可以传入一个能够完成全部这些工作的函数：

In [13]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

使用map是一种实现元素级转换以及其他数据清理工作的便捷方式。

## 替换值

利用fillna方法填充缺失数据可以看做值替换的一种特殊情况。前面已经看到，map可用于修改对象的数据子集，而replace则提供了一种实现该功能的更简单、更灵活的方式。我们来看看下面这个Series：

In [14]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

-999这个值可能是一个表示缺失数据的标记值。要将其替换为pandas能够理解的NA值，我们可以利用replace来产生一个新的Series（除非传入inplace=True）：

In [15]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

如果你希望一次性替换多个值，可以传入一个由待替换值组成的列表以及一个替换值：：

In [16]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

要让每个值有不同的替换值，可以传递一个替换列表：

In [17]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

传入的参数也可以是字典：

In [18]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

> 笔记：data.replace方法与data.str.replace不同，后者做的是字符串的元素级替换。我们会在后面学习Series的字符串方法。

## 重命名轴索引

跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新的不同标签的对象。轴还可以被就地修改，而无需新建一个数据结构。接下来看看下面这个简单的例子：

In [19]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four']
                   )

data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


跟Series一样，轴索引也有一个map方法：

In [22]:
transform = lambda x: x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [23]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


你可以将其赋值给index，这样就可以对DataFrame进行就地修改：

In [24]:
data.index = data.index.map(transform)

data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


如果想要创建数据集的转换版（而不是修改原始数据），比较实用的方法是rename：

In [27]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [28]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


特别说明一下，rename可以结合字典型对象实现对部分轴标签的更新：

In [26]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'}
           )

data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## 离散化和面元划分

为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）。假设有一组人员数据，而你希望将它们划分为不同的年龄组：

In [29]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

接下来将这些数据划分为“18到25”、“26到35”、“35到60”以及“60以上”几个面元。要实现该功能，你需要使用pandas的cut函数：

In [31]:
bins = [18, 25, 35, 60, 100]

cats = pd.cut(ages, bins)

cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

pandas返回的是一个特殊的 `Categorical对象`。结果展示了pandas.cut划分的面元。你可以将其看做一组表示面元名称的字符串。它的底层含有一个表示不同分类名称的类型数组，以及一个codes属性中的年龄数据的标签：

In [32]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [33]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [34]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

pd.value_counts(cats)是pandas.cut结果的面元计数。

跟“区间”的数学符号一样，圆括号表示开端，而方括号则表示闭端（包括）。哪边是闭端可以通过right=False进行修改：

In [35]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

你可 以通过传递一个列表或数组到labels，设置自己的面元名称：

In [41]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

bins = [18, 25, 35, 60, 100]

group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']


cats_v2 = pd.cut(ages, bins, labels=group_names)

cats_v2

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [42]:
pd.value_counts(cats_v2)

Youth         5
YoungAdult    3
MiddleAged    3
Senior        1
dtype: int64

如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算等长面元。下面这个例子中，我们将一些均匀分布的数据分成四组：

In [43]:
data = np.random.rand(20)

data

array([0.54661106, 0.21341118, 0.65048579, 0.34884671, 0.9600015 ,
       0.10419172, 0.73374222, 0.62416117, 0.82897023, 0.24581848,
       0.25113723, 0.96621713, 0.42638211, 0.74365619, 0.41152574,
       0.10811416, 0.3895286 , 0.0455443 , 0.31114258, 0.02729113])

In [45]:
res_v3 = pd.cut(data, 4, precision=2)

res_v3

[(0.5, 0.73], (0.026, 0.26], (0.5, 0.73], (0.26, 0.5], (0.73, 0.97], ..., (0.026, 0.26], (0.26, 0.5], (0.026, 0.26], (0.26, 0.5], (0.026, 0.26]]
Length: 20
Categories (4, interval[float64, right]): [(0.026, 0.26] < (0.26, 0.5] < (0.5, 0.73] < (0.73, 0.97]]

In [46]:
pd.value_counts(res_v3)

(0.026, 0.26]    7
(0.26, 0.5]      5
(0.73, 0.97]     5
(0.5, 0.73]      3
dtype: int64

选项precision=2，限定小数只有两位。

qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点。而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元：

In [47]:
data = np.random.randn(1000)  # Normally distributed

data

array([-1.10780132e+00, -1.84965723e-02, -8.51714256e-01,  3.02786669e-01,
        8.34704202e-01,  1.22747157e+00, -1.60465778e+00,  1.25436376e+00,
       -7.68407164e-01, -1.39019942e-01,  1.72546399e-01,  9.68792824e-01,
       -1.02136154e+00, -1.11638735e+00,  1.93257755e-01, -8.58587786e-01,
        1.13567150e+00,  5.45667879e-01,  9.10790043e-02, -1.33910985e+00,
       -1.27120220e-01,  4.71574570e-01,  7.88517195e-02,  2.03202875e+00,
        3.82826307e-01, -7.05548182e-01, -1.95197975e+00,  5.52617690e-01,
        1.34375651e+00,  1.67785442e-02, -2.24600928e-01, -6.15872946e-01,
        1.67037921e+00, -6.48359356e-03, -9.72379169e-02,  9.84265927e-01,
        2.76726982e+00, -7.39072712e-01, -9.73085135e-01,  8.88006510e-01,
        2.84805483e-01, -1.53314694e+00, -1.21367666e+00,  1.68075090e+00,
       -1.58652674e-01, -8.25579862e-01,  1.76249594e+00, -2.56894647e-01,
       -8.77476942e-01,  7.49414243e-02, -2.61678641e-02,  3.19212472e+00,
        1.17567781e+00, -

In [48]:
cats = pd.qcut(data, 4)  # Cut into quartiles

cats

[(-3.315, -0.656], (-0.656, -0.0133], (-3.315, -0.656], (-0.0133, 0.719], (0.719, 3.213], ..., (0.719, 3.213], (-3.315, -0.656], (-0.656, -0.0133], (0.719, 3.213], (-0.0133, 0.719]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.315, -0.656] < (-0.656, -0.0133] < (-0.0133, 0.719] < (0.719, 3.213]]

In [49]:
pd.value_counts(cats)

(-3.315, -0.656]     250
(-0.656, -0.0133]    250
(-0.0133, 0.719]     250
(0.719, 3.213]       250
dtype: int64

与cut类似，你也可以传递自定义的分位数（0到1之间的数值，包含端点）：

In [55]:
res_a = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

res_a

[(-1.176, -0.0133], (-1.176, -0.0133], (-1.176, -0.0133], (-0.0133, 1.274], (-0.0133, 1.274], ..., (-0.0133, 1.274], (-1.176, -0.0133], (-1.176, -0.0133], (-0.0133, 1.274], (-0.0133, 1.274]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.315, -1.176] < (-1.176, -0.0133] < (-0.0133, 1.274] < (1.274, 3.213]]

In [56]:
pd.value_counts(res_a)

(-1.176, -0.0133]    400
(-0.0133, 1.274]     400
(-3.315, -1.176]     100
(1.274, 3.213]       100
dtype: int64

本章稍后在讲解聚合和分组运算时会再次用到cut和qcut，因为这两个离散化函数对分位和分组分析非常重要。

## 检测和过滤异常值

过滤或变换异常值（outlier）在很大程度上就是运用数组运算。来看一个含有正态分布数据的DataFrame：

In [59]:
data = pd.DataFrame(np.random.randn(1000, 5))

data

,0,1,2,3,4
0,-0.628470,0.852790,-1.391426,0.349387,1.220137
1,0.058644,0.310716,-0.164380,0.024320,1.179610
2,1.254495,0.402902,-2.289132,1.794590,-0.105957
3,-0.146018,0.127235,-2.283615,-1.063468,-0.616641
4,-1.006175,-0.791468,0.252518,0.665419,-0.504912
...,...,...,...,...,...
995,-0.720880,-0.108241,-0.035614,0.207703,-0.254406
996,1.432456,-0.506653,-1.513577,-1.748046,-0.107797
997,2.028627,-2.256784,-0.139630,0.012115,0.191477
998,0.234811,-0.173545,0.143070,0.489465,-0.162274


In [58]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.023053,-0.017792,0.019038,-0.013172
std,0.961715,0.990408,1.042301,1.023584
min,-2.840085,-2.971167,-3.704109,-2.929153
25%,-0.706753,-0.683379,-0.686312,-0.698741
50%,0.001285,-0.048737,-0.007689,0.000962
75%,0.619917,0.655426,0.753050,0.699141
max,2.936533,2.938289,3.710361,4.072049


假设你想要找出某列中绝对值大小超过3的值：

In [61]:
col = data[2]

col[np.abs(col) > 3]

406    3.064622
563   -3.006721
Name: 2, dtype: float64

要选出全部含有“超过3或－3的值”的行，你可以在布尔型DataFrame中使用any方法：

In [62]:
data[(np.abs(data) > 3).any(1)]

/tmp/ipykernel_18158/2969116195.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  data[(np.abs(data) > 3).any(1)]


,0,1,2,3,4
79,-1.144499,0.219758,0.579791,-3.288305,0.775134
122,0.396242,3.219888,-1.771242,0.741403,-0.980629
130,1.013701,0.351426,0.754790,3.217495,-1.131902
406,-0.775349,1.332316,3.064622,-0.098900,1.107835
511,0.093836,0.191390,-1.139619,0.760228,3.115306
558,0.824484,0.415785,1.488446,-3.740257,-0.283510
563,1.849226,0.233756,-3.006721,-0.440746,0.410375
584,-1.575498,0.651002,1.233014,3.088325,0.616894
918,-1.931807,0.715558,-0.892657,0.523993,3.064280


根据这些条件，就可以对值进行设置。下面的代码可以将值限制在区间－3到3以内：

In [79]:
data[np.abs(data) > 3] = np.sign(data) * 3

a = data[data > 1]

a

,0,1,2,3,4
0,NaN,NaN,NaN,NaN,1.220137
1,NaN,NaN,NaN,NaN,1.179610
2,1.254495,NaN,NaN,1.79459,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN
996,1.432456,NaN,NaN,NaN,NaN
997,2.028627,NaN,NaN,NaN,NaN
998,NaN,NaN,NaN,NaN,NaN


In [64]:
data.describe()

,0,1,2,3,4
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.025961,0.014795,0.006197,0.052174,0.049874
std,0.971061,0.962089,0.998731,1.012816,0.980095
min,-2.956450,-2.836477,-3.000000,-3.000000,-2.999570
25%,-0.609803,-0.631368,-0.673561,-0.667661,-0.599227
50%,0.017840,0.033052,-0.002411,0.054808,0.019438
75%,0.665794,0.644878,0.669578,0.766049,0.694287
max,2.990743,3.000000,3.000000,3.000000,3.000000


根据数据的值是正还是负，np.sign(data)可以生成1和-1：

In [65]:
np.sign(data).head()

,0,1,2,3,4
0,-1.0,1.0,-1.0,1.0,1.0
1,1.0,1.0,-1.0,1.0,1.0
2,1.0,1.0,-1.0,1.0,-1.0
3,-1.0,1.0,-1.0,-1.0,-1.0
4,-1.0,-1.0,1.0,1.0,-1.0


## 排列和随机采样

利用`numpy.random.permutation`函数可以轻松实现对Series或DataFrame的列的排列工作（permuting，随机重排序）。通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组：

In [80]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))

df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [82]:
sampler = np.random.permutation(5)

sampler

array([3, 1, 2, 4, 0])

然后就可以在基于iloc的索引操作或take函数中使用该数组了：

In [83]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [84]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3


如果不想用替换的方式选取随机子集，可以在Series和DataFrame上使用sample方法：

In [90]:
df.sample(n=5)

,0,1,2,3
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7


要通过替换的方式产生样本（允许重复选择），可以传递replace=True到sample：

In [92]:
choices = pd.Series([5, 7, -1, 6, 4])

choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [94]:
draws = choices.sample(n=10, replace=True)

draws

2   -1
2   -1
2   -1
4    4
0    5
0    5
4    4
2   -1
2   -1
1    7
dtype: int64

## 计算指标/哑变量

另一种常用于统计建模或机器学习的转换方式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。

如果DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame（其值全为1和0）。pandas有一个get_dummies函数可以实现该功能（其实自己动手做一个也不难）。使用之前的一个DataFrame例子：

In [141]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


有时候，你可能想给指标DataFrame的列加上一个前缀，以便能够跟其他数据进行合并。get_dummies的prefix参数可以实现该功能：

In [139]:
df_key = df['key']

df_key

0    b
1    b
2    a
3    c
4    a
5    b
Name: key, dtype: object

In [140]:
# 原来的 a, b, c [a, b, c] 行中所处的位置信息
dummies = pd.get_dummies(df_key, prefix='key')

dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [101]:
df[['data1']]

,data1
0,0
1,1
2,2
3,3
4,4
5,5


In [100]:
df_with_dummy = df[['data1']].join(dummies)

df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0
